## Testovanie sentimentu pomocou ChatGPT – Zero-shot prompting

In [ ]:
# Inštalácia knižníc
!pip install openai pandas scikit-learn matplotlib seaborn

      
                                
import pandas as pd
import openai  # nainštalovanie openai, lebo Kaggle ho nemá predinštalovaný
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
                    

# Príprava dát 


In [ ]:
import pandas as pd
from datasets import load_dataset

#Načítanie slovenského datasetu
dataset = load_dataset("TUKE-KEMT/hate_speech_slovak")
df = pd.DataFrame(dataset["train"])

# Kontrola stĺpcov
# Očakáva sa buď 'text' alebo 'label' 0 = netoxický, 1 = toxický
print(df['label'].value_counts())

#Stratifikovaný výber do datasetu, sampling bez stratifikácie nefungoval, nevyvážený počet toxických komentárov

df_toxic = df[df['label'] == 1].sample(n=500, random_state=42)
df_nontoxic = df[df['label'] == 0].sample(n=500, random_state=42)

#Spojenie a premiešanie komentátrov
df_balanced = pd.concat([df_toxic, df_nontoxic]).sample(frac=1, random_state=42).reset_index(drop=True)

#Uloženie do Ccsv a použitie v ChatGPT zero-shot/few-shot notebookoch
df_balanced.to_csv("chatgpt_eval_1000_slovak.csv", index=False)
# ChatGPT má limit na počet tokenov – preto som zmenšil rozsah na 1000 komentárov
# pri velkom zaťažení odpovede padali alebo boli neúplné


print("Dataset pripravený 'chatgpt_eval_1000_slovak.csv'")


## 1. Načítanie dát

In [ ]:
# Dataset so 1000 komentármi (500 toxických + 500 netoxických)
df = pd.read_csv("/kaggle/input/chatgpt/chatgpt_eval_1000_slovak.csv")  # Komentáre z datasetu , ktorý bol upravený


## 2. Tvorba zero-shot promptu

In [ ]:
import os
import openai

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("OPENAI_API_KEY") # použivam systémovú premennú 

client = openai.OpenAI(api_key=api_key)

                              
# client = openai.OpenAI(api_key="")  # vlastný API kľúč 
              
                         
def create_prompt(text):
    return f"Urči sentiment nasledujúceho slovenského komentára ako pozitívny, negatívny alebo neutrálny:\n\n\"{text}\""
                            

## 3. Volanie OpenAI API

In [ ]:
results = []



for i, row in df.iterrows():
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": create_prompt(row["text"])}],
            temperature=0
        )                           
        sentiment = response.choices[0].message.content.strip()
    except Exception:
        sentiment = "CHYBA"
    results.append({"text": row["text"], "true_label": row["label"], "gpt_sentiment": sentiment})
    if i % 10 == 0:
        print(f"Spracovaných {i} komentárov...")

pd.DataFrame(results).to_csv("sentiment_results_test_1000.csv", index=False)
print("Výsledky uložené.")
                

## 4. Čistenie odpovedí a výpočet predikcií

In [ ]:
df = pd.read_csv("sentiment_results_test_1000.csv")

df["gpt_sentiment_clean"] = (
    df["gpt_sentiment"]
    .str.lower()
    .str.replace(".", "", regex=False)
    .str.strip()
)
                                  
df["predicted_label"] = df["gpt_sentiment_clean"].apply(lambda x: 1 if x == "negatívny" else 0) # Pomocou lambda funkcie priraďujem binárny štítok, ak výstup obsahuje slovo "negatívny", označím ho ako 1 (toxický)
df.to_csv("results_with_predictions_zero.csv", index=False)



## 5. Vyhodnotenie výsledkov

In [ ]:
y_true = df["true_label"]
y_pred = df["predicted_label"]

print("Klasifikačná správa:")
print(classification_report(y_true, y_pred, target_names=["Netoxický", "Toxický"]))

print(" Presnosť (accuracy):", round(accuracy_score(y_true, y_pred), 2))

print(" Precision:")
print("  Mikro:", round(precision_score(y_true, y_pred, average='micro'), 2))
print("  Makro:", round(precision_score(y_true, y_pred, average='macro'), 2))

print(" Recall:")
print(" - Mikro:", round(recall_score(y_true, y_pred, average='micro'), 2))
print(" - Makro:", round(recall_score(y_true, y_pred, average='macro'), 2))


print(" F1-score:")
print("  Mikro:", round(f1_score(y_true, y_pred, average='micro'), 2))
print("  Makro:", round(f1_score(y_true, y_pred, average='macro'), 2))


report_dict = classification_report(y_true, y_pred, target_names=["Netoxický", "Toxický"], output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv("classification_report_zeroshot.csv")



                    
report = classification_report(y_true, y_pred, target_names=["Netoxický", "Toxický"])
with open("classification_report_zeroshot.txt", "w") as f:
    f.write(report)

## 6. Vizualizácia 

In [ ]:
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Netoxický", "Toxický"], yticklabels=["Netoxický", "Toxický"])
plt.xlabel("Predikovaný štítok")
plt.ylabel("Skutočný štítok")
plt.title("Confusion matrix – Zero-shot prompting")
plt.tight_layout()
plt.savefig("confusion_matrix_zeroshot.png")
plt.show()

